In [1]:
import tensorflow as tf
import pickle
import sys
import matplotlib.pyplot as plt1 
import numpy as np
from IPython.display import Audio, display
import time
from sklearn.model_selection import train_test_split
from keras.layers import Input, Conv2D, Lambda, Dense, Flatten,MaxPooling2D, concatenate, Conv1D,Conv2D, Flatten, Reshape, Embedding, GRU, SpatialDropout1D, LSTM, Dropout, BatchNormalization
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.optimizers import SGD,Adam
from keras.losses import binary_crossentropy
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
from itertools import permutations
from sklearn.manifold import TSNE
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from scipy.stats import trim_mean

/Users/dweepa/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
data = pickle.load(open("../../CubeBio/Data/full","rb"))

In [3]:
data['id']=data['target']
data=data.drop(['target'],axis=1)

In [4]:
data.shape

(118050, 979)

In [5]:
full = pd.read_csv("../data/drug_class_identification/all3.csv")
full = full.dropna()
full['atc'] = full['atc'].apply(lambda x : x[0])
full.head()

,smiles,name,id,inchi_key,atc
75,NC12CC3CC(CC(C3)C1)C2,amantadine,BRD-K70330367,DKNWSYNQZKUICI-UHFFFAOYSA-N,N
291,CS(=O)(=O)OCCCCOS(C)(=O)=O,busulfan,BRD-K23204545,COVZYZSDYWQREU-UHFFFAOYSA-N,L
322,CCCC(C)(COC(N)=O)COC(=O)NC(C)C,carisoprodol,BRD-A99939097,OFZCIYFFPZCNJE-UHFFFAOYSA-N,M
491,CCN(CC)C(=O)N1CCN(C)CC1,diethylcarbamazine,BRD-K45542189,RCKMWOKWVGPNJF-UHFFFAOYSA-N,P
510,CCN(CC)C(=S)SSC(=S)N(CC)CC,disulfiram,BRD-K32744045,AUZONCFQVSMFAP-UHFFFAOYSA-N,P


In [6]:
result = pd.merge(data, full, how="inner").drop(['id','smiles','name','inchi_key'],axis=1)

In [7]:
result.dropna()
result.head()

,780,7849,6193,23,9552,387,10921,10285,533,6194,...,11000,6915,6253,7264,5467,2767,23038,57048,79716,atc
0,-0.066358,-1.245058,0.078668,-0.788176,-1.129236,0.407757,0.292397,0.479977,-0.223936,-1.141854,...,-0.784618,-0.982621,-0.670931,0.981703,0.883668,0.202455,0.429206,0.312411,-0.133215,D
1,-0.066358,-1.245058,0.078668,-0.788176,-1.129236,0.407757,0.292397,0.479977,-0.223936,-1.141854,...,-0.784618,-0.982621,-0.670931,0.981703,0.883668,0.202455,0.429206,0.312411,-0.133215,G
2,-0.209095,0.176860,-0.720512,-0.793518,0.720299,-0.555583,-0.395229,-1.252304,-2.462850,-0.185532,...,0.235278,-0.136517,0.864514,-0.098884,-1.099747,-3.044908,-0.146705,0.942604,0.028090,D
3,-0.209095,0.176860,-0.720512,-0.793518,0.720299,-0.555583,-0.395229,-1.252304,-2.462850,-0.185532,...,0.235278,-0.136517,0.864514,-0.098884,-1.099747,-3.044908,-0.146705,0.942604,0.028090,G
4,1.160403,0.857906,-2.114255,0.041883,0.818861,0.302385,-0.586580,-0.774723,1.168153,-1.972200,...,1.197446,0.940610,0.434476,1.187713,-0.335655,0.262674,1.139491,0.858688,1.959727,D


In [8]:
y = result['atc']
X = result.drop(['atc'],axis=1)
X = np.asarray(X)
X.shape

(49373, 978)

In [9]:
# Define our own plot function
def scatter(x, y, subtitle=None):
    le = LabelEncoder()
    labels = le.fit_transform(y)

    # We choose a color palette with seaborn.
    palette = np.array(sns.color_palette("hls", 14))

    # We create a scatter plot.
    f = plt.figure(figsize=(8, 8))
    ax = plt.subplot(aspect='equal')
    sc = ax.scatter(x[:,0], x[:,1], lw=0, s=40,
                    c=palette[labels.astype(np.int)])
    plt.xlim(-25, 25)
    plt.ylim(-25, 25)
    ax.axis('off')
    ax.axis('tight')

    # We add the labels for each digit.
    txts = []
    for i in range(14):
        # Position of each label.
        xtext, ytext = trim_mean(x[labels == i, :], axis=0, proportiontocut=0.2)
        letter = le.inverse_transform([i])[0]
        txt = ax.text(xtext, ytext, str(letter), fontsize=24)
        txt.set_path_effects([
            PathEffects.Stroke(linewidth=5, foreground="w"),
            PathEffects.Normal()])
        txts.append(txt)
        
    if subtitle != None:
        plt.suptitle(subtitle)
        
    plt.savefig(subtitle)

In [10]:
x_train, x_test,y_train,y_test = train_test_split(X,y)

x_train_flat = x_train.reshape(-1,978)
x_test_flat = x_test.reshape(-1,978)

tsne = TSNE()
train_tsne_embeds = tsne.fit_transform(x_train_flat[:500])
scatter(train_tsne_embeds, y_train[:500], "Samples from Training Data LINCS")

eval_tsne_embeds = tsne.fit_transform(x_test_flat[:500])
scatter(eval_tsne_embeds, y_test[:500], "Samples from Validation Data LINCS")

/Users/dweepa/anaconda/lib/python3.6/site-packages/scipy/stats/stats.py:2831: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.mean(atmp[sl], axis=axis)
/Users/dweepa/anaconda/lib/python3.6/site-packages/scipy/stats/stats.py:2831: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.mean(atmp[sl], axis=axis)


In [23]:
def triplet_loss(y_true, y_pred, alpha = 0.4):
    print('y_pred.shape = ',y_pred)
    
    total_lenght = y_pred.shape.as_list()[-1]
    
    anchor = y_pred[:,0:int(total_lenght*1/3)]
    positive = y_pred[:,int(total_lenght*1/3):int(total_lenght*2/3)]
    negative = y_pred[:,int(total_lenght*2/3):int(total_lenght*3/3)]

    # distance between the anchor and the positive
    pos_dist = K.sum(K.square(anchor-positive),axis=1)

    # distance between the anchor and the negative
    neg_dist = K.sum(K.square(anchor-negative),axis=1)

    # compute loss
    basic_loss = pos_dist-neg_dist+alpha
    loss = K.maximum(basic_loss,0.0)
 
    return loss

def baseNetwork():    
        model = Sequential()
        for i in range(10)
            model.add(Dense(100, activation='relu'))
            model.add(Dropout(0.9))
        model.add(Dense(14, activation='softmax'))
        return model
    
anchor_input = Input((978,1, ), name='anchor_input')
positive_input = Input((978,1, ), name='positive_input')
negative_input = Input((978,1, ), name='negative_input')

# Shared embedding layer for positive and negative items
Shared_DNN = baseNetwork()


encoded_anchor = Shared_DNN(anchor_input)
encoded_positive = Shared_DNN(positive_input)
encoded_negative = Shared_DNN(negative_input)


merged_vector = concatenate([encoded_anchor, encoded_positive, encoded_negative], axis=-1, name='merged_layer')

model = Model(inputs=[anchor_input,positive_input, negative_input], outputs=merged_vector)
model.compile(loss=triplet_loss, optimizer='adam')
model.summary()

SyntaxError: invalid syntax (<ipython-input-23-80f100e6c83c>, line 24)

In [26]:
def generateTriplet(x,y,testsize=0.2,ap_pairs=10,an_pairs=10):
    data_xy = tuple([x,y])

    trainsize = 1-testsize

    triplet_train_pairs = []
    triplet_test_pairs = []
    for data_class in sorted(set(data_xy[1])):

        same_class_idx = np.where((data_xy[1] == data_class))[0]
        diff_class_idx = np.where(data_xy[1] != data_class)[0]
        A_P_pairs = random.sample(list(permutations(same_class_idx,2)),k=ap_pairs) #Generating Anchor-Positive pairs
        Neg_idx = random.sample(list(diff_class_idx),k=an_pairs)
        

        #train
        A_P_len = len(A_P_pairs)
        Neg_len = len(Neg_idx)
        for ap in A_P_pairs[:int(A_P_len*trainsize)]:
            Anchor = data_xy[0][ap[0]]
            Positive = data_xy[0][ap[1]]
            for n in Neg_idx:
                Negative = data_xy[0][n]
                triplet_train_pairs.append([Anchor,Positive,Negative])               
        #test
        for ap in A_P_pairs[int(A_P_len*trainsize):]:
            Anchor = data_xy[0][ap[0]]
            Positive = data_xy[0][ap[1]]
            for n in Neg_idx:
                Negative = data_xy[0][n]
                triplet_test_pairs.append([Anchor,Positive,Negative])    
                
    return np.array(triplet_train_pairs), np.array(triplet_test_pairs)

In [27]:
X_train, X_test = generateTriplet(X[:50],y[:50])

In [28]:
Anchor = X_train[:,0,:].reshape(-1,978,1)
Positive = X_train[:,1,:].reshape(-1,978,1)
Negative = X_train[:,2,:].reshape(-1,978,1)
Anchor_test = X_test[:,0,:].reshape(-1,978,1)
Positive_test = X_test[:,1,:].reshape(-1,978,1)
Negative_test = X_test[:,2,:].reshape(-1,978,1)

Y_dummy = np.empty((Anchor.shape[0],300))
Y_dummy2 = np.empty((Anchor_test.shape[0],1))

model.fit([Anchor,Positive,Negative],y=Y_dummy,validation_data=([Anchor_test,Positive_test,Negative_test],Y_dummy2), epochs=10)

Train on 160 samples, validate on 40 samples
Epoch 1/10
160/160 [==============================] - 94s 585ms/step - loss: 0.4005 - val_loss: 0.4000
Epoch 2/10
160/160 [==============================] - 75s 469ms/step - loss: 0.4000 - val_loss: 0.4000
Epoch 3/10
160/160 [==============================] - 78s 485ms/step - loss: 0.3998 - val_loss: 0.4000
Epoch 4/10
160/160 [==============================] - 74s 460ms/step - loss: 0.3995 - val_loss: 0.4000
Epoch 5/10
160/160 [==============================] - 75s 469ms/step - loss: 0.3994 - val_loss: 0.4000
Epoch 6/10
160/160 [==============================] - 73s 453ms/step - loss: 0.3994 - val_loss: 0.4000
Epoch 7/10
160/160 [==============================] - 73s 456ms/step - loss: 0.3984 - val_loss: 0.3999
Epoch 8/10
160/160 [==============================] - 71s 443ms/step - loss: 0.3982 - val_loss: 0.3999
Epoch 9/10
160/160 [==============================] - 78s 490ms/step - loss: 0.3988 - val_loss: 0.3999
Epoch 10/10
160/160 [=======

In [29]:
trained_model = Model(inputs=anchor_input, outputs=encoded_anchor)

In [35]:
tsne = TSNE()
X_train_trm = trained_model.predict(x_train[:100].reshape(-1,978,1))
X_test_trm = trained_model.predict(x_test[:100].reshape(-1,978,1))
train_tsne_embeds = tsne.fit_transform(X_train_trm)
eval_tsne_embeds = tsne.fit_transform(X_test_trm)

ValueError: Error when checking input: expected anchor_input to have 3 dimensions, but got array with shape (100, 978)

In [33]:
scatter(train_tsne_embeds, y_train[:500], "Training Data After TNN")
scatter(eval_tsne_embeds, y_test[:500], "Validation Data After TNN")

/Users/dweepa/anaconda/lib/python3.6/site-packages/scipy/stats/stats.py:2831: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.mean(atmp[sl], axis=axis)
